In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [ ]:
data_dir = Path('../data')
network_name = 'IEEE39'
if network_name == 'IEEE39':
    suffix = 'modified_network'
    conditions = (
        # 'general_load',
        'const_P',
        'const_Z'
    )
    step_SM_name = 'G_07'
    H = 3.771, 8.771
    fname = '39_Bus_New_England_System_AC_TF_-6.0_2.0_100_0.01.npz'
    cmap_name = 'tab10'
elif network_name == 'WSCC':
    # conditions = (
    #     'loads_A_B_C_const_I',
    #     'loads_A_B_C_const_Z',
    #     'loads_A_B_C_const_P',
    #     'loads_A_B_C_const_P_stable',
    #     'load_A_const_P_loads_B_C_const_Z',
    #     'load_B_const_P_loads_A_C_const_Z',
    #     'load_C_const_P_loads_A_B_const_Z',
    #     'loads_A_B_const_P_load_C_const_Z',
    #     'loads_A_C_const_P_load_B_const_Z',
    #     'loads_B_C_const_P_load_A_const_Z',
    #     'loads_B_C_D_const_P_load_A_const_Z',
    #     'loads_A_B_C_const_P_load_D_const_Z',
    # )    
    suffix = ''
    conditions = (
        'loads_A_B_C_const_Z',
        'loads_A_B_C_const_P',
        'loads_A_B_C_const_P_stable',
    )    
    step_SM_name = 'G2'
    H = 3.33, #6.66
    fname = '9_Bus_WSCC_AC_TF_-6.0_2.0_100_0.01_Load_A_Load_B_Load_C.npz'
    cmap_name = 'tab20'

In [ ]:
data = {
    cond: {
        h: np.load(data_dir / network_name / cond / suffix / f'{step_SM_name}_h_{h:g}' / fname,
                   allow_pickle=True) for h in H
    }
    for cond in conditions
}

In [ ]:
Δ = 0.35
input_loads = data[conditions[0]][H[0]]['input_loads'].tolist()
dB = 0
SM_to_exclude = 'G 01'
ndx, = np.where(D['SM_names'] != SM_to_exclude)
for input_load in input_loads:
    for cond in conditions:
        fig, ax = plt.subplots(2, 1, figsize=(6, 5), sharex=True)
        cmap = plt.get_cmap('tab10')
        for i, h in enumerate(H):
            D = data[cond][h]
            jdx, = np.where(D['input_loads'] == input_load)
            # keep in the right order
            kdx = np.array([i for SM_name in D['SM_names']
                            for i, var_name in enumerate(D['var_names'])
                            if SM_name in var_name and SM_name != SM_to_exclude])
            F, TF = D['F'], D['TF']
            TF_COI = (TF[:, jdx, kdx] * D['H_SM'][ndx] * D['S_SM'][ndx]).sum(axis=1) / (D['H_SM'][ndx] @ D['S_SM'][ndx])
            col = cmap(i)
            light_col = np.array([np.array(col[:3]) + Δ, np.ones(3)]).min(axis=0)
            y = np.abs(TF[:, jdx, kdx])
            if dB > 0:
                y = dB * np.log10(y)
            ax[0].plot(F, y, color=light_col, lw=0.75)
            y = np.abs(TF_COI)
            if dB > 0:
                y = dB * np.log10(y)
            ax[0].plot(F, y, color=col, lw=1.5)
            y = np.unwrap(np.angle(TF[:, jdx, kdx]), axis=0)
            if network_name == 'IEEE39' and ndx.size == 10:
                ax[1].plot(F, y[:, 1:], color=light_col, lw=0.75)
                ax[1].plot(F, y[:, 0], color=[.5,.5,.5], lw=1.5)
            else:
                ax[1].plot(F, y, color=light_col, lw=0.75)
            ax[1].plot(F, np.unwrap(np.angle(TF_COI)), color=col, lw=1.5)
        ax[0].set_xscale('log')
        ax[1].set_xscale('log')
        ax[-1].set_xlabel('Frequency [Hz]')
        if dB > 0:
            ax[0].set_ylabel(f'|TF| [dB{dB}]')
        else:
            ax[0].set_ylabel('|TF|')            
        ax[1].set_ylabel('Phase [rad]')
        sns.despine()
        fig.tight_layout()
        fname = 'TF_input_{}_{}{}.pdf'.\
            format(input_load.replace(' ', '_'), cond, f'_dB{dB}' if dB > 0 else '')
        outfile = data_dir / network_name / fname
        plt.savefig(outfile)
        plt.close(fig)

In [ ]:
if network_name == 'IEEE39':
    h = 3.771
    ctrl = data['const_Z'][h]
    other = data['const_P'][h]
elif network_name == 'WSCC':
    h = 3.33
    ctrl = data['loads_A_B_C_const_Z'][h]
    other = data['loads_A_B_C_const_P_stable'][h]

eig_ctrl, _ = np.linalg.eig(ctrl['A'])
eig_other, _ = np.linalg.eig(other['A'])
assert eig_ctrl.size == eig_other.size, 'The systems do not have the same number of state variables'
idx = np.argsort(eig_ctrl.real)[::-1]
for i, (ec, eo) in enumerate(zip(eig_ctrl[idx], eig_other[idx])):
    ec_re, ec_im = ec.real, ec.imag
    eo_re, eo_im = eo.real, eo.imag
    print('[{:3d}] {:8.3f} {} j{:5.3f} {:8.3f} {} j{:5.3f}'.format(
        i + 1,
        ec_re,
        '+' if ec_im >= 0 else '-',
        abs(ec_im),
        eo_re,
        '+' if eo_im >= 0 else '-',
        abs(eo_im),
    ))